In [2]:
%matplotlib inline
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import datetime as dt

def plot_stock(ticker, years):
    end = dt.datetime.now()
    start = end - dt.timedelta(days=365 * int(years))
    
    df = yf.download(ticker, start=start, end=end)
    if df.empty:
        print('No data found. Please check ticker symbol.')
        return

    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_slope'] = df['MACD'].diff()

    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    df['SMA20'] = df['Close'].rolling(window=20).mean()
    df['STD20'] = df['Close'].rolling(window=20).std()
    df['UpperBand'] = df['SMA20'] + 2 * df['STD20']
    df['LowerBand'] = df['SMA20'] - 2 * df['STD20']
    latest_close = df['Close'][ticker].iloc[-1]
    upper = df['UpperBand'].iloc[-1]
    lower = df['LowerBand'].iloc[-1]
    sma = df['SMA20'].iloc[-1]
    band_width = abs(upper - lower)/sma

    df['Close'].plot(title=f"{ticker} Adjusted Close ({years} year)", figsize=(10, 6))
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.grid(True)
    plt.show()
    stock = yf.Ticker(ticker)
    info = stock.info
    name = info.get('longName', 'N/A')
    pe = info.get('trailingPE', None)
    div_yield = info.get('dividendYield', None)
    market_cap = info.get('marketCap', None)
    pb = info.get('priceToBook', None)
    roe = info.get('returnOnEquity', None)
    dte = info.get('debtToEquity', None)
    beta = info.get('beta', None)
    forpe = info.get('forwardPE', None)
    rsi = df['RSI'].iloc[-1]
    print(f'{name} Valuation Insights')
    print('')
    
    if pe is not None:
        print(f'  - Trailing P/E Ratio: {round(pe, 2)}')
        if pe < 15:
            print(f'  {name} has a low Trailing P/E Ratio, {ticker} might be undervalued.')
        elif pe > 30:
            print(f'  WARNING! {name} has a high Trailing P/E Ratio, {ticker} might be overvalued or a growth stock.')
        elif pe >=15:
            print(f'  {name} has a relatively average Trailing P/E Ratio, {ticker} is likely neither overvalued nor undervalued.')
    else:
        print(f'  - Trailing P/E Ratio not available.')
    print('')
    
    if roe is not None:
        print(f'  - Return on Equity: {round(roe*100, 2)}%')
        if roe < 0.08:
            print(f'  {name} has a low Return on Equity, implying that {ticker} is inefficient at generating profits.')
        elif roe > 0.17:
            print(f'  {name} has a high Return on Equity, implying that {ticker} is efficient at generating profits.')
        else:
            print(f'  {name} has a Return on Equity around market average, implying that {ticker} is average at generating profits.')
    else:
        print(f'  - Return on Equity not available.')

    print('')
    print(f'{name} Risk Metrics')
    print('')
    
    if div_yield is not None:
        print(f'  - Dividend Yield: {round(div_yield, 3)}%')
        if div_yield > 5:
            print(f'  WARNING! {name} has a very high Dividend Yield, {ticker} might be high risk, but could also offer high immediate returns.')
        elif div_yield >= 3:
            print(f'  {name} has a high Dividend Yield, {ticker} could offer attractive immediate returns.')
        elif 0 < div_yield < 3:
            print(f'  {name} has a safe Dividend Yield, {ticker} is likely a sound investment.')
        elif div_yield == 0:
            print(f'  {name} does not pay Dividends, and is likely focusing on growth. Investing in {ticker} will not provide you with a stream of income, which is fine as long as the stock value increases.')
    else:
        print(f'  - Dividend Yield not available.')
    print('')
    
    if dte is not None:
        print(f'  - Debt to Equity Ratio: {round(dte/100, 2)}. Compare to industry peers for a fair assesment.')
        if dte < 50:
            print(f'  {name} has a low Debt to Equity Ratio, indicating that {ticker} prioritizes financial stability and conservative growth.')
        elif dte > 130:
            print(f'  {name} has a high Debt to Equity Ratio, indicating that {ticker} relies heavily on debt financing, which may support aggressive growth but also increases financial risk.')
        else:
            print(f'  {name} has a moderate Debt to Equity Ratio, indicating that {ticker} is neither overly aggressive nor conservative with their debt leveraging.')
    else:
        print(f'  - Debt to Equity Ratio not available.')
    print('')
    
    if beta is not None:
        print(f'  - Beta: {round(beta, 2)}')
        if beta > 1.2:
            print(f'  {name} is more volatile than the market, amplifying market moves by a factor of {round(beta, 2)}')
        elif 0 <= beta < 0.8:
            print(f'  {name} is less volatile than the market, amplifying market moves by a factor of {round(beta, 2)}')
        elif beta < 0:
            print(f'  {ticker} moves inversely to the market')
        else:
            print(f'  {ticker} moves relatively in line with the market. The volatility of {name} is about average.')
    else:
        print(f'  - Beta not available')
    print('')
    
    print(f'  - Bollinger Band Width: {round(band_width, 4)}')
    if band_width > 0.12:
        print(f'  {name} has loose Bollinger Bands, indicating high volatility.')
    elif band_width < 0.05:
        print(f'  {name} has tight Bollinger Bands, indicating low volatility.')
    else:
        print(f'  {name} has a moderate Bollinger Band width, indicating average volatility.')
    
    print('')
    print(f'{name} Future Projections')
    print('')
    
    if forpe is not None:
        print(f'  - Forward P/E Ratio: {round(forpe, 2)}')
        if pe is not None:
            if forpe < 0.9*pe:
                print(f'  The Forward P/E Ratio of {name} is significantly lower than the Trailing P/E Ratio, indicating analysts are expecting improved price action over the next year.')
            elif 0.9*forpe > pe:
                print(f'  The Forward P/E Ratio of {name} is significantly higher than the Trailing P/E Ratio, indicating analysts are expecting bearish price action over the next year.')
            else:
                print(f'  The Forward P/E Ratio of {name} is similar to the Trailing P/E Ratio, which indicates neither bearish nor bullish price action.')
    else:
        print(f'  - Forward P/E Ratio not available.')
    print('')
    
    if df['MACD'].iloc[-1] < df['Signal'].iloc[-1] and df['MACD_slope'].iloc[-1] > 0 and df['MACD'].iloc[-1] < 0 and df['Signal'].iloc[-1] < 0:
        print(f'  The Moving Average Convergence Divergence (MACD) indicates {name} is likely around its trough and approaching a local bull run.')
    elif df['MACD'].iloc[-1] > df['Signal'].iloc[-1] and df['MACD_slope'].iloc[-1] < 0 and df['MACD'].iloc[-1] > 0 and df['Signal'].iloc[-1] > 0:
        print(f'  The Moving Average Convergence Divergence (MACD) indicates {name} is likely around its peak and approaching a local bear run.')
    elif df['MACD'].iloc[-1] < df['Signal'].iloc[-1] and df['MACD_slope'].iloc[-1] <= 0:
        print(f'  The Moving Average Convergence Divergence (MACD) indicates {name} is likely in a short term bear market.')
    elif df['MACD'].iloc[-1] > df['Signal'].iloc[-1] and df['MACD_slope'].iloc[-1] >= 0:
        print(f'  The Moving Average Convergence Divergence (MACD) indicates {name} is likely in a short term bull market.')
    else:
        print(f'  The Moving Average Convergence Divergence (MACD) does not indicate anything clear regarding {name}')
    print('')
    
    if rsi is not None:
        print(f'  - Relative Strength Index: {round(rsi, 2)}')
        if rsi > 70:
            print(f'  The RSI suggests {name} may be near a peak and could experience a price pullback soon.')
        elif rsi < 30:
            print(f'  The RSI suggests {name} may be near a trough and could experience a rebound soon.')
        elif 50 < rsi <= 70 and rsi > df['RSI'].iloc[-6] > df['RSI'].iloc[-11]:
            print(f'  The RSI suggests {name} is currently in a short term bull market. If the RSI approaches 70, the bull market is likely nearing its end.')
        elif 30 <= rsi < 50 and rsi < df['RSI'].iloc[-6] < df['RSI'].iloc[-11]:
            print(f'  The RSI suggests {name} is currently in a short term bear market. If the RSI approaches 30, the bear market is likely nearing its end.')
        else:
            print(f'  The RSI does not indicate a strong directional signal. Watch for moves toward 70 (overbought) or 30 (oversold).')
    else:
        print(f'  - Relative Strength Index not available.')
    print('')

    print(f'  - Bollinger Bands: Upper = {round(upper, 2)}, SMA20 = {round(sma, 2)}, Lower = {round(lower, 2)}')
    print(f'  - Latest Close: {round(latest_close, 2)}')
    if latest_close > upper:
        print(f'  The price is above the upper Bollinger Band, suggesting {name} may be overbought or experiencing strong momentum. {ticker} is likely due for a bear market.')
    elif latest_close < lower:
        print(f'  The price is below the lower Bollinger Band, suggesting {name} may be oversold or due for a rebound.')
    elif lower <= latest_close <= upper:
        print(f'  The price is within the Bollinger Bands, no strong signal is indicated. Watch for a move to either Band.')


interact(plot_stock, ticker=widgets.Text(value='AAPL'), years=widgets.IntSlider(min=1, max=50, step=1, value=5))


interactive(children=(Text(value='AAPL', description='ticker'), IntSlider(value=5, description='years', max=50…

<function __main__.plot_stock(ticker, years)>